In [1]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST

In [2]:
API_KEY = "PK1KAWB732Z95VFMPIEB"
API_SECRET = "lRxdiVh4jOoyZQvgUdqplHNuQZWI9apIPIXPaoaM"
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER": True
}

In [4]:


class MLTrader(Strategy):
    def initialize(self, symbol:str="SPY", cash_at_risk:float = .5):
        self.cash_at_risk = cash_at_risk
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
    
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)

        return cash, last_price, quantity
    
    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        if cash > last_price:
            if self.last_trade == None:
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",
                    take_profit_price=last_price*1.20,
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order)
                self.last_trade = "buy"


brokers = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=brokers, parameters={"symbol": "SPY", 
                                                                "cash_at_risk":.5})

start_date = datetime(2023,12,15)
end_date = datetime(2023,12,31)

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol": "SPY", 
                "cash_at_risk":.5}
)

INFO:backtest_stats:Starting backtest...


Starting backtest for MLTrader...
Progress || 100.00%  [Elapsed: 0:00:01 ETA: 0:00:00] Portfolio Val: 100,944.04
Creating trades plot...

Creating indicators plot...

Creating tearsheet...


{'cagr': 0.25708368926531966,
 'volatility': 0.028190598556940217,
 'sharpe': 7.270285016328554,
 'max_drawdown': {'drawdown': 0.0027690901105407078,
  'date': Timestamp('2023-12-21 09:30:00-0500', tz='America/New_York')},
 'romad': 92.84049236488013,
 'total_return': 0.00944036000000037}

## ML Components for news (Sentimental Analysis)

In [ ]:
# pip install alpaca-trade-api==3.1.1